In [3]:
import numpy as np
from numpy import load

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, Normalizer

In [4]:
lista_archivos_guardar = ["etiqueta_fruta","imagen_fruta"]
test_imagenes  = np.load('./Input/test_imagenes.npy')
test_etiqueta  = np.load('./Input/test_etiqueta.npy')
etiqueta_fruta = np.load('./Input/etiqueta_fruta.npy')
imagen_fruta   = np.load('./Input/imagen_fruta.npy')


In [5]:
imagen_fruta.shape,etiqueta_fruta.shape,test_etiqueta.shape,test_imagenes.shape

((60486, 35, 35, 3), (60486,), (20618,), (20618, 35, 35, 3))

### Vamos a hacer una estadarizacion y PCA

In [6]:
estandar = StandardScaler()
listad_imagenes = [imagenes.flatten() for imagenes in imagen_fruta]
images_estandar = estandar.fit_transform(listad_imagenes)

In [25]:
normalizar = Normalizer()

imagenes_normalizadas = normalizar.fit_transform(e )

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
data_2d = pca.fit_transform(images_estandar)

In [1]:
data_2d

NameError: name 'data_2d' is not defined

In [20]:
imagenes_normalizadas

(60486, 2352)

In [ ]:
pca = 